Realizamos los imports necesarios e iniciamos una SparkSession

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` 
import $ivy.`sh.almond::almond-spark:0.4.0`

import org.apache.spark.sql.{NotebookSparkSession, SparkSession}
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._

val spark = NotebookSparkSession
      .builder()
      .config("spark.sql.join.preferSortMergeJoin", false)
      .config("spark.sql.shuffle.partitions", 64)
      .master("local[*]")
      .getOrCreate()

import spark.implicits._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

Logger.getRootLogger().setLevel(Level.ERROR)

Loading spark-stubs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/07/18 20:16:09 INFO SparkContext: Running Spark version 2.4.5
22/07/18 20:16:10 INFO SparkContext: Submitted application: 6217d6f8-c8b2-49f9-846a-f46644380e83
22/07/18 20:16:10 INFO SecurityManager: Changing view acls to: Alvaro
22/07/18 20:16:10 INFO SecurityManager: Changing modify acls to: Alvaro
22/07/18 20:16:10 INFO SecurityManager: Changing view acls groups to: 
22/07/18 20:16:10 INFO SecurityManager: Changing modify acls groups to: 
22/07/18 20:16:10 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(Alvaro); groups with view permissions: Set(); users  with modify permissions: Set(Alvaro); groups with modify permissions: Set()
22/07/18 20:16:12 INFO Utils: Successfully started service 'sparkDriver' on port 49535.
22/07/18 20:16:12 INFO SparkEnv: Registering MapOutputTracker
22/07/18 20:16:12 INFO SparkEnv: Registering BlockManagerMa

22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/com/lihaoyi/utest_2.12/0.6.4/utest_2.12-0.6.4.jar at spark://LAPTOP-30N4Q54J.mshome.net:49535/jars/utest_2.12-0.6.4.jar with timestamp 1658168173647
22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/io/get-coursier/coursier-cache_2.12/1.1.0-M13-1/coursier-cache_2.12-1.1.0-M13-1.jar at spark://LAPTOP-30N4Q54J.mshome.net:49535/jars/coursier-cache_2.12-1.1.0-M13-1.jar with timestamp 1658168173648
22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/io/get-coursier/coursier-core_2.12/1.1.0-M13-1/coursier-core_2.12-1.1.0-M13-1.jar at spark://LAPTOP-30N4Q54J.mshome.net:49535/jars/coursier-core_2.12-1.1.0-M13-1.jar with timestamp 1658168173650
22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/App

...

22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/apache/curator/curator-client/2.6.0/curator-client-2.6.0.jar at spark://LAPTOP-30N4Q54J.mshome.net:49535/jars/curator-client-2.6.0.jar with timestamp 1658168173877
22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/xerial/snappy/snappy-java/1.1.7.3/snappy-java-1.1.7.3.jar at spark://LAPTOP-30N4Q54J.mshome.net:49535/jars/snappy-java-1.1.7.3.jar with timestamp 1658168173878
22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/javax/validation/validation-api/1.1.0.Final/validation-api-1.1.0.Final.jar at spark://LAPTOP-30N4Q54J.mshome.net:49535/jars/validation-api-1.1.0.Final.jar with timestamp 1658168173879
22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/

22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/json4s/json4s-ast_2.12/3.5.3/json4s-ast_2.12-3.5.3.jar at spark://LAPTOP-30N4Q54J.mshome.net:49535/jars/json4s-ast_2.12-3.5.3.jar with timestamp 1658168173913
22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/apache/avro/avro-mapred/1.8.2/avro-mapred-1.8.2-hadoop2.jar at spark://LAPTOP-30N4Q54J.mshome.net:49535/jars/avro-mapred-1.8.2-hadoop2.jar with timestamp 1658168173915
22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/apache/parquet/parquet-encoding/1.10.1/parquet-encoding-1.10.1.jar at spark://LAPTOP-30N4Q54J.mshome.net:49535/jars/parquet-encoding-1.10.1.jar with timestamp 1658168173916
22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/h

22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/io/argonaut/argonaut_2.12/6.2.3/argonaut_2.12-6.2.3.jar at spark://LAPTOP-30N4Q54J.mshome.net:49535/jars/argonaut_2.12-6.2.3.jar with timestamp 1658168173956
22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/eclipse/jetty/jetty-server/9.4.15.v20190215/jetty-server-9.4.15.v20190215.jar at spark://LAPTOP-30N4Q54J.mshome.net:49535/jars/jetty-server-9.4.15.v20190215.jar with timestamp 1658168173958
22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/sh/almond/ammonite-spark_2.12/0.4.0/ammonite-spark_2.12-0.4.0.jar at spark://LAPTOP-30N4Q54J.mshome.net:49535/jars/ammonite-spark_2.12-0.4.0.jar with timestamp 1658168173959
22/07/18 20:16:13 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Cours

Spark UI

import $ivy.$                                   

import $ivy.$                              


import org.apache.spark.sql.{NotebookSparkSession, SparkSession}

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.types._


spark: SparkSession = org.apache.spark.sql.SparkSession@70d6b2e7
import spark.implicits._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}



In [51]:
val data: DataFrame = spark.read.option("multiline", "true").json("D:/TFGAlvaroSanchez/data/*.json")

json at cmd50.sc:1

1 / 1

data: DataFrame = [e: string, evap: string ... 42 more fields]

In [54]:
data.show()

show at cmd53.sc:1

1 / 1

show at cmd53.sc:1

1 / 1

+-----+------+-------+-------+----+----------+----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+-----+-----+-------+-------+-------+-----+-----+------------+-----+-----+------+-------------+------+-------------+------------+-------------+------+------+------+------+-----+------+-----+---------------+-----+
|    e|  evap|  fecha|    glo|  hr|indicativo|inso|n_cub|n_des|n_fog|n_gra|n_llu|n_nie|n_nub|n_tor|np_001|np_010|np_100|np_300|nt_00|nt_30|nv_0050|nv_0100|nv_1000|nw_55|nw_91|       p_max|p_mes|p_sol| q_mar|        q_max| q_med|        q_min|      ta_max|       ta_min|ti_max|tm_max|tm_mes|tm_min|ts_20|ts_min|w_med|        w_racha|w_rec|
+-----+------+-------+-------+----+----------+----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+-----+-----+-------+-------+-------+-----+-----+------------+-----+-----+------+-------------+------+-------------+------------+-------------+------+------+------+------+-----+------+-----+------

### Obtencion de la fecha y del valor maximo de temperatura

Con lo siguiente cambiamos el tipo de datos de la columna ta_max

val data1 = data.withColumn("ta_max", $"ta_max".substr(0,4).cast(IntegerType))

In [4]:
import org.apache.spark.sql.functions.{min, max, desc}

data.withColumn("ta_max", $"ta_max".substr(0,4).cast(IntegerType))
    .select($"fecha", $"ta_max" as "temperatura maxima")
    .orderBy($"ta_max".desc)
    .show(1)

show at cmd3.sc:6

8 / 8

+------+------------------+
| fecha|temperatura maxima|
+------+------------------+
|2021-8|                42|
+------+------------------+
only showing top 1 row



import org.apache.spark.sql.functions.{min, max, desc}



### Obtencion del mes, año, id estacion y provincia donde ocurren la stemperaturas mas altas en 2021

In [5]:
val data2021 : DataFrame = spark.read.option("multiline", "true").json("D:/TFGAlvaroSanchez/data/*2021.json")

json at cmd4.sc:1

1 / 1

data2021: DataFrame = [e: string, evap: string ... 42 more fields]

In [6]:
data2021.printSchema()

root
 |-- e: string (nullable = true)
 |-- evap: string (nullable = true)
 |-- fecha: string (nullable = true)
 |-- glo: string (nullable = true)
 |-- hr: string (nullable = true)
 |-- indicativo: string (nullable = true)
 |-- inso: string (nullable = true)
 |-- n_cub: string (nullable = true)
 |-- n_des: string (nullable = true)
 |-- n_fog: string (nullable = true)
 |-- n_gra: string (nullable = true)
 |-- n_llu: string (nullable = true)
 |-- n_nie: string (nullable = true)
 |-- n_nub: string (nullable = true)
 |-- n_tor: string (nullable = true)
 |-- np_001: string (nullable = true)
 |-- np_010: string (nullable = true)
 |-- np_100: string (nullable = true)
 |-- np_300: string (nullable = true)
 |-- nt_00: string (nullable = true)
 |-- nt_30: string (nullable = true)
 |-- nv_0050: string (nullable = true)
 |-- nv_0100: string (nullable = true)
 |-- nv_1000: string (nullable = true)
 |-- nw_55: string (nullable = true)
 |-- nw_91: string (nullable = true)
 |-- p_max: string (nullable 

In [7]:
data2021.select($"ta_min", $"indicativo", $"fecha")
    .filter($"indicativo".equalTo("2916A"))
    .show()

show at cmd6.sc:2

1 / 1

show at cmd6.sc:2

4 / 4

+------------+----------+-------+
|      ta_min|indicativo|  fecha|
+------------+----------+-------+
|    -6.6(12)|     2916A| 2021-1|
|    -1.7(22)|     2916A| 2021-2|
|    -2.6(21)|     2916A| 2021-3|
|    -1.5(17)|     2916A| 2021-4|
|     2.0(02)|     2916A| 2021-5|
|     6.1(28)|     2916A| 2021-6|
|     6.6(13)|     2916A| 2021-7|
|     7.9(01)|     2916A| 2021-8|
|     5.9(29)|     2916A| 2021-9|
|     3.2(24)|     2916A|2021-10|
|    -3.4(28)|     2916A|2021-11|
|    -3.1(03)|     2916A|2021-12|
|-6.6(12/ene)|     2916A|2021-13|
+------------+----------+-------+



Usamos `.df` para darle nombre a las columnas

In [8]:
val ids : DataFrame = spark.read.option("delimiter", ";").csv("D:/TFGAlvaroSanchez/data/aemetID.csv")
    .toDF("provincia", "indicativo", "ubicacion")

csv at cmd7.sc:1

1 / 1

ids: DataFrame = [provincia: string, indicativo: string ... 1 more field]

In [9]:
ids.show()
ids.printSchema()

show at cmd8.sc:1

1 / 1

+----------------+----------+--------------------+
|       provincia|indicativo|           ubicacion|
+----------------+----------+--------------------+
|        A Coruna|      1351|     Estaca de Bares|
|        A Coruna|     1363X|           As Pontes|
|        A Coruna|      1387|            A Coruna|
|        A Coruna|     1387E| A Coruna Aeropuerto|
|        A Coruna|     1390X|Carballo, Depuradora|
|        A Coruna|      1393|          Cabo Vilan|
|        A Coruna|      1399|            Vimianzo|
|        A Coruna|      1400|            Fisterra|
|        A Coruna|     1406X|           Mazaricos|
|        A Coruna|     1410X|             Sobrado|
|        A Coruna|      1428|Santiago de Compo...|
|        A Coruna|     1435C|                Noia|
|        A Coruna|     1442U|               Boiro|
|        A Coruna|     1473A|              Padron|
|        A Coruna|     1475X|Santiago de Compo...|
|        A Coruna|     1476R|Rois, Casas do Porto|
|Alacant/Alicante|     7244X|  

In [10]:
val joindata : DataFrame = data2021.join(ids, "indicativo")

joindata: DataFrame = [indicativo: string, e: string ... 44 more fields]

In [11]:
joindata.withColumn("ta_max", $"ta_max".substr(0,4).cast(IntegerType))
    .select($"fecha".substr(6,2) as "mes", $"fecha".substr(0,4) as "año", $"ta_max" as "temperatura maxima", $"indicativo", $"provincia")
    .filter(!($"mes".equalTo("13")))
    .orderBy($"ta_max".desc)
    .show()

run at ThreadPoolExecutor.java:1128

5 / 5

show at cmd10.sc:4

1 / 1

+---+----+------------------+----------+--------------------+
|mes| año|temperatura maxima|indicativo|           provincia|
+---+----+------------------+----------+--------------------+
|  8|2021|                42|      3129|              Madrid|
|  8|2021|                41|     2916A|           Salamanca|
|  7|2021|                40|      3129|              Madrid|
|  7|2021|                37|     2916A|           Salamanca|
|  8|2021|                36|     C449C|Santa Cruz de Ten...|
|  6|2021|                35|      3129|              Madrid|
|  6|2021|                34|     2916A|           Salamanca|
|  9|2021|                33|      1387|            A Coruna|
|  9|2021|                33|      3129|              Madrid|
|  9|2021|                33|     2916A|           Salamanca|
|  7|2021|                32|     C449C|Santa Cruz de Ten...|
|  8|2021|                32|     0201D|           Barcelona|
|  5|2021|                32|      3129|              Madrid|
|  5|202

### Obtencion del mes, año y provincia donde se encuntra la temperatura mas baja durante verano (junio, julio y agosto)

In [12]:
joindata.withColumn("ta_min", $"ta_min".substr(0,4).cast(IntegerType))
    .withColumn("fecha", $"fecha".cast(DateType)).printSchema

root
 |-- indicativo: string (nullable = true)
 |-- e: string (nullable = true)
 |-- evap: string (nullable = true)
 |-- fecha: date (nullable = true)
 |-- glo: string (nullable = true)
 |-- hr: string (nullable = true)
 |-- inso: string (nullable = true)
 |-- n_cub: string (nullable = true)
 |-- n_des: string (nullable = true)
 |-- n_fog: string (nullable = true)
 |-- n_gra: string (nullable = true)
 |-- n_llu: string (nullable = true)
 |-- n_nie: string (nullable = true)
 |-- n_nub: string (nullable = true)
 |-- n_tor: string (nullable = true)
 |-- np_001: string (nullable = true)
 |-- np_010: string (nullable = true)
 |-- np_100: string (nullable = true)
 |-- np_300: string (nullable = true)
 |-- nt_00: string (nullable = true)
 |-- nt_30: string (nullable = true)
 |-- nv_0050: string (nullable = true)
 |-- nv_0100: string (nullable = true)
 |-- nv_1000: string (nullable = true)
 |-- nw_55: string (nullable = true)
 |-- nw_91: string (nullable = true)
 |-- p_max: string (nullable = 

Para quedarnos unicamente con la temperatura ya que este valor viene precedido del dia que ocurrio de la siguiente manera: temperatura(dia)

In [13]:
import org.apache.spark.sql.functions.split

joindata.withColumn("ta_min", split($"ta_min", "\\("))
    .select($"ta_min"(0) as "val")
    .show()

run at ThreadPoolExecutor.java:1128

5 / 5

show at cmd12.sc:4

1 / 1

+----+
| val|
+----+
| 2.3|
| 6.3|
| 7.6|
| 9.1|
|12.1|
|14.6|
|13.8|
|10.3|
| 6.5|
| 6.2|
| 5.9|
| 6.7|
| 2.3|
| 2.0|
| 4.8|
| 5.3|
|12.5|
|15.7|
|18.4|
|18.2|
+----+
only showing top 20 rows



import org.apache.spark.sql.functions.split



In [46]:
joindata.withColumn("ta_min", split($"ta_min", "\\("))
    .withColumn("ta_min", $"ta_min"(0).cast(IntegerType))
    .withColumn("fecha", $"fecha".cast(DateType))
    .select(func.month($"fecha") as "mes", func.year($"fecha") as "año", $"ta_min" as "temperatura minima", $"provincia")  
    .filter($"mes" > 5 && $"mes" < 9)
    .orderBy($"temperatura minima".asc)
    .show()

run at ThreadPoolExecutor.java:1128

5 / 5

show at cmd45.sc:6

1 / 1

+---+----+------------------+--------------------+
|mes| año|temperatura minima|           provincia|
+---+----+------------------+--------------------+
|  6|2021|                 6|           Salamanca|
|  7|2021|                 6|           Salamanca|
|  8|2021|                 7|           Salamanca|
|  7|2021|                10|              Madrid|
|  6|2021|                10|            A Coruna|
|  6|2021|                11|              Madrid|
|  8|2021|                12|              Madrid|
|  7|2021|                13|            A Coruna|
|  8|2021|                14|            A Coruna|
|  6|2021|                17|Santa Cruz de Ten...|
|  6|2021|                17|           Barcelona|
|  8|2021|                18|           Barcelona|
|  7|2021|                18|           Barcelona|
|  7|2021|                19|Santa Cruz de Ten...|
|  8|2021|                20|Santa Cruz de Ten...|
+---+----+------------------+--------------------+



### Numero de meses con temperaturas maximas >30º por provincia

In [43]:
joindata
    .withColumn("ta_max", split($"ta_max", "\\("))
    .withColumn("ta_max", $"ta_max"(0).cast(IntegerType))
    .withColumn("fecha", $"fecha".cast(DateType))
    .filter($"ta_max" > 30 && !func.isnull($"fecha"))//> "0000-00-00")
    .groupBy("provincia")
    .count
    .show()

run at ThreadPoolExecutor.java:1128

5 / 5

show at cmd42.sc:6

1 / 1

show at cmd42.sc:6

...

show at cmd42.sc:6

20 / 20

show at cmd42.sc:6

39 / 39

+--------------------+-----+
|           provincia|count|
+--------------------+-----+
|            A Coruna|    1|
|           Salamanca|    5|
|              Madrid|    5|
|           Barcelona|    1|
|Santa Cruz de Ten...|    2|
+--------------------+-----+



### Uso de graficos mostrando la precipitacion mensual por meses y años en Vitigudino(Salamanca)

In [16]:
val vitData : DataFrame = spark.read.option("multiline", "true").json("D:/TFGAlvaroSanchez/data/2916A*.json")

json at cmd15.sc:1

1 / 1

vitData: DataFrame = [e: string, fecha: string ... 34 more fields]

Obtenemos los datos que queremos mostrar

In [44]:
val vitDataRequired : DataFrame = vitData
    .withColumn("fecha", $"fecha".cast(DateType))
    .withColumn("p_mes", $"p_mes".cast(IntegerType))
    .filter(!func.isnull($"fecha") && $"p_mes" >= 0)
    .select($"fecha", $"p_mes")
    .orderBy($"fecha".asc)

vitDataRequired: DataFrame = [fecha: date, p_mes: int]

In [45]:
vitDataRequired.show()

show at cmd44.sc:1

4 / 4

+----------+-----+
|     fecha|p_mes|
+----------+-----+
|2018-01-01|   53|
|2018-02-01|   52|
|2018-03-01|  172|
|2018-04-01|  102|
|2018-05-01|   76|
|2018-06-01|   49|
|2018-07-01|   29|
|2018-08-01|    0|
|2018-09-01|    7|
|2018-10-01|   26|
|2018-11-01|  147|
|2018-12-01|   31|
|2019-01-01|   36|
|2019-02-01|   13|
|2019-03-01|   22|
|2019-04-01|   97|
|2019-05-01|   11|
|2019-06-01|   38|
|2019-07-01|   17|
|2019-08-01|   22|
+----------+-----+
only showing top 20 rows



In [19]:
val vitDataRequired_f : Seq[String] = vitDataRequired.select($"fecha").as[String].collect.toSeq
val vitDataRequired_p : Seq[Int] = vitDataRequired.select($"p_mes").as[Int].collect.toSeq

collect at cmd18.sc:1

4 / 4

collect at cmd18.sc:1

4 / 4

collect at cmd18.sc:1

...

collect at cmd18.sc:2

4 / 4

collect at cmd18.sc:2

46 / 46

vitDataRequired_f: Seq[String] = WrappedArray(
  "2018-01-01",
  "2018-02-01",
  "2018-03-01",
  "2018-04-01",
  "2018-05-01",
  "2018-06-01",
  "2018-07-01",
  "2018-08-01",
  "2018-09-01",
  "2018-10-01",
  "2018-11-01",
  "2018-12-01",
  "2019-01-01",
  "2019-02-01",
  "2019-03-01",
  "2019-04-01",
  "2019-05-01",
  "2019-06-01",
  "2019-07-01",
  "2019-08-01",
  "2019-09-01",
  "2019-10-01",
  "2019-11-01",
  "2019-12-01",
  "2020-01-01",
  "2020-02-01",
  "2020-03-01",
  "2020-04-01",
  "2020-05-01",
  "2020-06-01",
  "2020-07-01",
  "2020-08-01",
  "2020-09-01",
  "2020-10-01",
  "2020-11-01",
  "2020-12-01",
  "2021-01-01",
  "2021-02-01",
...
vitDataRequired_p: Seq[Int] = WrappedArray(
  53,
  52,
  172,
  102,
  76,
  49,
  29,
  0,
  7,
  26,
  147,
  31,
  36,
  13,
  22,
  97,
  11,
  38,
  17,
  22,
  24,
  62,
  98,
  119,
  56,
  8,
  48,
  97,
  23,
  2,
  0,
  24,
  35,
  103,
  56,
  49,
  46,
  117,
...

Realizamos los imports necesarios para las graficas

In [20]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells


In [21]:
val trace = Seq(
    Scatter(
        vitDataRequired_f,
        vitDataRequired_p,
        fill = Fill.ToZeroY,
        marker = Marker(
            color = Color.RGBA(55, 128, 191, 0.6)
        )
    )
)


val layout = Layout(
    title = "Precipitacion mensual (mm)",
    paper_bgcolor =  Color.RGBA(245, 246, 249, 1),
    plot_bgcolor = Color.RGBA(245, 246, 249, 1),
)
plot(trace, layout)

trace: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("Precipitacion mensual (mm)"),
...
res20_2: String = "plot-6ed8e753-25e2-4624-af9a-7ca7e7121058"

### Leer los datos de manera casteada mediante un schema

In [47]:
val schema = StructType(
                Array(
                    StructField("fecha", DateType, true) 
                )
            )

schema: StructType = StructType(StructField("fecha", DateType, true, {}))

In [49]:
val dt: DataFrame = spark.read.schema(schema).option("multiline", "true").json("D:/TFGAlvaroSanchez/data/*.json")

dt: DataFrame = [fecha: date]

In [50]:
dt.show()

show at cmd49.sc:1

1 / 1

show at cmd49.sc:1

4 / 4

show at cmd49.sc:1

3 / 3

+-----+
|fecha|
+-----+
| null|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
+-----+



### Mostrando graficamente la precipitacion mensual en diferentes estaciones meterologicas

In [29]:
val data2021 = spark.read.option("multiline", "true").json("D:/TFGAlvaroSanchez/data/*-2021.json")

json at cmd28.sc:1

1 / 1

data2021: DataFrame = [e: string, evap: string ... 42 more fields]

In [30]:
data2021.write.partitionBy("indicativo").saveAsTable("indicativos")

: 